<a href="https://colab.research.google.com/github/Emilyfroy/Israel-Imports-2025-Analysis/blob/main/Project_num1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Emily Froymovich**

# Project Objective

**The goal** of this project is to analyze Israel’s import patterns in 2024 using detailed line-level import transaction data. The analysis focuses on achieving the following:


1.   Identify the major imported products and leading product categories, based on financial value and volume.

2.   Map the key origin countries from which Israel imports goods, and evaluate the degree of dependence on these countries.

3.   Understand how different customs procedures (Government Procedure Types) impact the structure, flow, and value of imports.

4.   Conduct an in-depth case study of a strategic product group — crude petroleum, including:


*   price-per-unit analysis
*   treatment of multiple measurement units
*   segmentation by exporting country
*   identifying cost differences, seasonal patterns, and supply concentration.

5.   Demonstrate advanced data capabilities, including data cleaning, merging external classification sources (HS codes), exploratory data analysis (EDA), outlier detection, and advanced visualizations using Plotly.

In [299]:
import pandas as pd
import numpy as np
import plotly.express as px

In [300]:
file_path = "/content/import_datafile_2024.csv"
df = pd.read_csv(file_path)

print("צורת הטבלה:", df.shape)
df.head()

/tmp/ipython-input-418241223.py:2: DtypeWarning:

Columns (3,4,5,7,8,10,12,15,19) have mixed types. Specify dtype option on import or set low_memory=False.



צורת הטבלה: (2097416, 23)


,Year,Month,Origin_Country,CustomsItem_2_Digits,CustomsItem_8_Digits,Exempt_CustomsItem,CustomsHouse,Quantity,Quantity_MeasurementUnitID,Quantity_MeasurementUnitName,...,NISCurrencyAmount,CurrencyCode,AutonomyTypeID,AutonomyType,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName,IsPreferenceDocument,GeneralCustomsTax,PurchaseTax,VAT
0,2024,1,AD,90,90321021,NaN,Ben Gurion Airport,1.0,1,Each,...,36.28,EUR,NaN,Israel,יבוא מסחרי,CommercialImport,1,0.0,10.0,10.0
1,2024,1,AE,12,12119090,NaN,Allenby,129.76,6,Kilogram,...,7325.88,USD,1.0,Judea and Samaria,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,0,0.0,0.0,1194.0
2,2024,1,AE,20,20029019,NaN,Allenby,22733.0,6,Kilogram,...,106848.06,USD,1.0,Judea and Samaria,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,0,16595.0,0.0,19914.0
3,2024,1,AE,21,21011200,NaN,Haifa,12913.0,6,Kilogram,...,49442.98,USD,1.0,Judea and Samaria,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,0,0.0,0.0,8280.0
4,2024,1,AE,21,21069099,NaN,Ashdod,861.0,6,Kilogram,...,26241.29,USD,1.0,Judea and Samaria,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,0,1067.0,0.0,4439.0


In [301]:
#Printing columns table
def print_column_info(df):
  for col in df.columns:
      print(col)
print_column_info(df)

Year
Month
Origin_Country
CustomsItem_2_Digits
CustomsItem_8_Digits
Exempt_CustomsItem
CustomsHouse
Quantity
Quantity_MeasurementUnitID
Quantity_MeasurementUnitName
TradeAgreementName
TermsOfSale
IsTradeAgreementWithQuota
NISCurrencyAmount
CurrencyCode
AutonomyTypeID
AutonomyType
GovernmentProcedureTypeName
GovernmentProcedureTypeEnglishName
IsPreferenceDocument
GeneralCustomsTax
PurchaseTax
VAT


In [302]:
#Selecting relevant columns

cols = [
    "Month",
    "Origin_Country",
    "CustomsItem_2_Digits",
    "CustomsItem_8_Digits",
    "CustomsHouse",
    "Quantity",
    "Quantity_MeasurementUnitName",
    "NISCurrencyAmount",
    "GovernmentProcedureTypeName",
    "GovernmentProcedureTypeEnglishName"
]

df = df[cols]
print(df.shape)
df.head()


(2097416, 10)


,Month,Origin_Country,CustomsItem_2_Digits,CustomsItem_8_Digits,CustomsHouse,Quantity,Quantity_MeasurementUnitName,NISCurrencyAmount,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName
0,1,AD,90,90321021,Ben Gurion Airport,1.0,Each,36.28,יבוא מסחרי,CommercialImport
1,1,AE,12,12119090,Allenby,129.76,Kilogram,7325.88,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy
2,1,AE,20,20029019,Allenby,22733.0,Kilogram,106848.06,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy
3,1,AE,21,21011200,Haifa,12913.0,Kilogram,49442.98,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy
4,1,AE,21,21069099,Ashdod,861.0,Kilogram,26241.29,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy


In [303]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097416 entries, 0 to 2097415
Data columns (total 10 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Month                               int64  
 1   Origin_Country                      object 
 2   CustomsItem_2_Digits                object 
 3   CustomsItem_8_Digits                object 
 4   CustomsHouse                        object 
 5   Quantity                            object 
 6   Quantity_MeasurementUnitName        object 
 7   NISCurrencyAmount                   float64
 8   GovernmentProcedureTypeName         object 
 9   GovernmentProcedureTypeEnglishName  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 160.0+ MB


In [304]:
#Converting to numeric data type
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097416 entries, 0 to 2097415
Data columns (total 10 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Month                               int64  
 1   Origin_Country                      object 
 2   CustomsItem_2_Digits                object 
 3   CustomsItem_8_Digits                object 
 4   CustomsHouse                        object 
 5   Quantity                            float64
 6   Quantity_MeasurementUnitName        object 
 7   NISCurrencyAmount                   float64
 8   GovernmentProcedureTypeName         object 
 9   GovernmentProcedureTypeEnglishName  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 160.0+ MB


In [305]:
df['Month'].value_counts()

,count
Month,
3,186258
7,186111
12,182563
9,180109
5,177549
8,174237
2,173457
6,169860
10,168585


Using another data file from the same source to match the product description

In [306]:
file_path = "/content/Israel Customs Tariff and Purchase Tax.csv"
hs= pd.read_csv(file_path)

print("צורת הטבלה:",hs.shape)
hs.head()

צורת הטבלה: (12821, 17)


,CustomsBookTypeID,CustomsItem_2_Digits,CustomsItem_4_Digits,CustomsItem_6_Digits,CustomsItemFullClassification,HierarchicLocation,GoodsDescription,FullGoodsDescription,MeasurementUnitID,MeasurementUnitDescription,CustomsItemEndDate,CustomsTariff,CustomsTariffEndDate,PurchaseTaxTariff,PurchaseTaxTariffEndDate,PercentageCapAdditionTariff,MaslulMeasurementUnit
0,1,1,100,10000,0100000000,Chapter,Chapter 1- Live animals,Chapter 1- Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,101,10100,0101000000,Heading,"Live horses, asses, mules (hinnies) -","Live horses, asses, mules (hinnies) -",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,101,10120,0101200000,Sub Heading,:-horses,"Live horses, asses, mules (hinnies) - >...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,101,10121,0101210000/2,Israel Sub Heading,Pure-bred breeding animals,"Live horses, asses, mules (hinnies) - >...",1.0,Each,NaN,Tax Free,NaN,Tax Free,NaN,NaN,6.0
4,1,1,101,10129,0101290000/5,Israel Sub Heading,Other,"Live horses, asses, mules (hinnies) - >...",1.0,Each,NaN,Tax Free,NaN,Tax Free,NaN,NaN,6.0


In [307]:
hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821 entries, 0 to 12820
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CustomsBookTypeID              12821 non-null  int64  
 1   CustomsItem_2_Digits           12821 non-null  int64  
 2   CustomsItem_4_Digits           12821 non-null  int64  
 3   CustomsItem_6_Digits           12821 non-null  int64  
 4   CustomsItemFullClassification  12821 non-null  object 
 5   HierarchicLocation             12821 non-null  object 
 6   GoodsDescription               12814 non-null  object 
 7   FullGoodsDescription           12821 non-null  object 
 8   MeasurementUnitID              8928 non-null   float64
 9   MeasurementUnitDescription     8928 non-null   object 
 10  CustomsItemEndDate             99 non-null     object 
 11  CustomsTariff                  1634 non-null   object 
 12  CustomsTariffEndDate           7458 non-null  

In [308]:
hs["CustomsItem_8_Digits"] = (
    hs["CustomsItemFullClassification"]
    .astype(str)       # Convert to string
    .str.zfill(8)      # Make sure there are 8 digits with leading zeros
    .str[:8]           # Take the digits 1-8
)
hs.head()

,CustomsBookTypeID,CustomsItem_2_Digits,CustomsItem_4_Digits,CustomsItem_6_Digits,CustomsItemFullClassification,HierarchicLocation,GoodsDescription,FullGoodsDescription,MeasurementUnitID,MeasurementUnitDescription,CustomsItemEndDate,CustomsTariff,CustomsTariffEndDate,PurchaseTaxTariff,PurchaseTaxTariffEndDate,PercentageCapAdditionTariff,MaslulMeasurementUnit,CustomsItem_8_Digits
0,1,1,100,10000,0100000000,Chapter,Chapter 1- Live animals,Chapter 1- Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01000000
1,1,1,101,10100,0101000000,Heading,"Live horses, asses, mules (hinnies) -","Live horses, asses, mules (hinnies) -",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01010000
2,1,1,101,10120,0101200000,Sub Heading,:-horses,"Live horses, asses, mules (hinnies) - >...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01012000
3,1,1,101,10121,0101210000/2,Israel Sub Heading,Pure-bred breeding animals,"Live horses, asses, mules (hinnies) - >...",1.0,Each,NaN,Tax Free,NaN,Tax Free,NaN,NaN,6.0,01012100
4,1,1,101,10129,0101290000/5,Israel Sub Heading,Other,"Live horses, asses, mules (hinnies) - >...",1.0,Each,NaN,Tax Free,NaN,Tax Free,NaN,NaN,6.0,01012900


In [309]:
#Step 1: Create a uniform key column in both tables
# Preparing the key in the import table
df["key_8_digits"] = (
    pd.to_numeric(df["CustomsItem_8_Digits"], errors="coerce")  # המרה למספר
    .astype("Int64")                                            # מספר שלם
    .astype("string")                                           # מחרוזת
)

# Preparing the key in the goods classification book
hs["key_8_digits"] = (
    pd.to_numeric(hs["CustomsItem_8_Digits"], errors="coerce")
    .astype("Int64")
    .astype("string")
    .str.zfill(8))



In [310]:
#Step 2: Check if there is any overlap between the codes
common_keys = df["key_8_digits"].isin(
    hs["key_8_digits"]
).mean()

print("אחוז הרשומות שיש להן התאמה בספר:", common_keys)


אחוז הרשומות שיש להן התאמה בספר: 0.9902360809682009


In [311]:
#Create a matching mask
mask_match = df["key_8_digits"].isin(hs["key_8_digits"])

In [312]:
#Retrieve the records that do not match
df_no_match = df[~mask_match].copy()

In [313]:
#Check what the problem is
df_no_match["key_8_digits"].value_counts().head(20)

,count
key_8_digits,
39204990,718
9011100,707
4069099,635
9012100,587
2023000,369
7132000,328
9023000,321
3044100,305
8081000,303


In [314]:
df_no_match[
    ["CustomsItem_2_Digits", "CustomsItem_8_Digits", "Origin_Country", "Quantity", "NISCurrencyAmount"]
].head(20)

,CustomsItem_2_Digits,CustomsItem_8_Digits,Origin_Country,Quantity,NISCurrencyAmount
21,8,8041010,AE,50016.00,242920.41
30,2,2013000,AR,659719.26,15575590.65
31,2,2023000,AR,23572.00,176635.34
32,2,2023000,AR,13872.45,101942.53
33,2,2062900,AR,427.00,1882.70
34,3,3036600,AR,26798.00,210125.54
35,7,7132000,AR,25000.00,109437.59
69,7,7135000,AU,47525.00,124641.35
125,1,1061100,CA,1.00,364.51
126,7,7132000,CA,99700.00,287521.10


In [315]:
codes_to_remove = df_no_match["CustomsItem_8_Digits"].unique()

work_df = df[~df["CustomsItem_8_Digits"].isin(codes_to_remove)].copy()
work_df.head()

,Month,Origin_Country,CustomsItem_2_Digits,CustomsItem_8_Digits,CustomsHouse,Quantity,Quantity_MeasurementUnitName,NISCurrencyAmount,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName,key_8_digits
0,1,AD,90,90321021,Ben Gurion Airport,1.00,Each,36.28,יבוא מסחרי,CommercialImport,90321021
1,1,AE,12,12119090,Allenby,129.76,Kilogram,7325.88,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,12119090
2,1,AE,20,20029019,Allenby,22733.00,Kilogram,106848.06,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,20029019
3,1,AE,21,21011200,Haifa,12913.00,Kilogram,49442.98,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,21011200
4,1,AE,21,21069099,Ashdod,861.00,Kilogram,26241.29,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,21069099


In [316]:
print("צורת הטבלה אחרי מחיקה:", work_df.shape)

# Make sure there are no more mismatched codes
(work_df["CustomsItem_8_Digits"].isin(df_no_match["CustomsItem_8_Digits"])).sum()


צורת הטבלה אחרי מחיקה: (2076937, 11)


np.int64(0)

In [317]:
print("אחוז התאמות:", work_df["key_8_digits"].isin(hs["key_8_digits"]).mean())


אחוז התאמות: 1.0


In [318]:
hs_small = hs[[
    "key_8_digits",
    "CustomsItem_2_Digits",
    "CustomsItem_6_Digits",
    "GoodsDescription",
    "FullGoodsDescription"
]]
hs_small.head()

,key_8_digits,CustomsItem_2_Digits,CustomsItem_6_Digits,GoodsDescription,FullGoodsDescription
0,01000000,1,10000,Chapter 1- Live animals,Chapter 1- Live animals
1,01010000,1,10100,"Live horses, asses, mules (hinnies) -","Live horses, asses, mules (hinnies) -"
2,01012000,1,10120,:-horses,"Live horses, asses, mules (hinnies) - >..."
3,01012100,1,10121,Pure-bred breeding animals,"Live horses, asses, mules (hinnies) - >..."
4,01012900,1,10129,Other,"Live horses, asses, mules (hinnies) - >..."


In [319]:
df_merged = work_df.merge(
    hs_small,
    on="key_8_digits",
    how="left"
)
df_merged.head()

,Month,Origin_Country,CustomsItem_2_Digits_x,CustomsItem_8_Digits,CustomsHouse,Quantity,Quantity_MeasurementUnitName,NISCurrencyAmount,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName,key_8_digits,CustomsItem_2_Digits_y,CustomsItem_6_Digits,GoodsDescription,FullGoodsDescription
0,1,AD,90,90321021,Ben Gurion Airport,1.00,Each,36.28,יבוא מסחרי,CommercialImport,90321021,90,903210,"----OF THE KIND USED IN MOTOR VEHICLES, OTHER ...",Automatic regulating or controlling instrument...
1,1,AE,12,12119090,Allenby,129.76,Kilogram,7325.88,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,12119090,12,121190,---OTHERS,Plants and parts of plants (including seeds an...
2,1,AE,20,20029019,Allenby,22733.00,Kilogram,106848.06,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,20029019,20,200290,OTHERS,Tomatoes prepared or preserved otherwise than ...
3,1,AE,21,21011200,Haifa,12913.00,Kilogram,49442.98,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,21011200,21,210112,"-- Preparations with a basis of extracts, esse...","Extracts, essences and concentrates, of coffee..."
4,1,AE,21,21069099,Ashdod,861.00,Kilogram,26241.29,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,21069099,21,210690,----OTHERS,Food preparations not elsewhere specified or i...


In [320]:
print_column_info(df_merged)

Month
Origin_Country
CustomsItem_2_Digits_x
CustomsItem_8_Digits
CustomsHouse
Quantity
Quantity_MeasurementUnitName
NISCurrencyAmount
GovernmentProcedureTypeName
GovernmentProcedureTypeEnglishName
key_8_digits
CustomsItem_2_Digits_y
CustomsItem_6_Digits
GoodsDescription
FullGoodsDescription


In [321]:
work_df = df_merged[['Month','Origin_Country','CustomsItem_8_Digits','CustomsHouse','Quantity','Quantity_MeasurementUnitName','NISCurrencyAmount','GovernmentProcedureTypeName','GovernmentProcedureTypeEnglishName','GoodsDescription','FullGoodsDescription']]
work_df.head()

,Month,Origin_Country,CustomsItem_8_Digits,CustomsHouse,Quantity,Quantity_MeasurementUnitName,NISCurrencyAmount,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName,GoodsDescription,FullGoodsDescription
0,1,AD,90321021,Ben Gurion Airport,1.00,Each,36.28,יבוא מסחרי,CommercialImport,"----OF THE KIND USED IN MOTOR VEHICLES, OTHER ...",Automatic regulating or controlling instrument...
1,1,AE,12119090,Allenby,129.76,Kilogram,7325.88,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,---OTHERS,Plants and parts of plants (including seeds an...
2,1,AE,20029019,Allenby,22733.00,Kilogram,106848.06,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,OTHERS,Tomatoes prepared or preserved otherwise than ...
3,1,AE,21011200,Haifa,12913.00,Kilogram,49442.98,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,"-- Preparations with a basis of extracts, esse...","Extracts, essences and concentrates, of coffee..."
4,1,AE,21069099,Ashdod,861.00,Kilogram,26241.29,יבוא מסחרי-אוטונומיה,CommercialImportAutonomy,----OTHERS,Food preparations not elsewhere specified or i...


In [322]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083657 entries, 0 to 2083656
Data columns (total 11 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Month                               int64  
 1   Origin_Country                      object 
 2   CustomsItem_8_Digits                object 
 3   CustomsHouse                        object 
 4   Quantity                            float64
 5   Quantity_MeasurementUnitName        object 
 6   NISCurrencyAmount                   float64
 7   GovernmentProcedureTypeName         object 
 8   GovernmentProcedureTypeEnglishName  object 
 9   GoodsDescription                    object 
 10  FullGoodsDescription                object 
dtypes: float64(2), int64(1), object(8)
memory usage: 174.9+ MB


In [323]:
work_df["Quantity"] = pd.to_numeric(work_df["Quantity"], errors="coerce")
work_df["CustomsItem_8_Digits"] = pd.to_numeric(work_df["CustomsItem_8_Digits"], errors="coerce")
work_df.info()

/tmp/ipython-input-807558009.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083657 entries, 0 to 2083656
Data columns (total 11 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Month                               int64  
 1   Origin_Country                      object 
 2   CustomsItem_8_Digits                int64  
 3   CustomsHouse                        object 
 4   Quantity                            float64
 5   Quantity_MeasurementUnitName        object 
 6   NISCurrencyAmount                   float64
 7   GovernmentProcedureTypeName         object 
 8   GovernmentProcedureTypeEnglishName  object 
 9   GoodsDescription                    object 
 10  FullGoodsDescription                object 
dtypes: float64(2), int64(2), object(7)
memory usage: 174.9+ MB


/tmp/ipython-input-807558009.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [324]:
work_df['CustomsHouse'].value_counts()

,count
CustomsHouse,
Ben Gurion Airport,1203588
Ashdod,651001
Haifa,189758
Center,18991
Allenby,10090
Jordan River,6815
Nitsana terminal,2173
Rabin Crossing,952
Eilat,289


In [325]:
work_df['GovernmentProcedureTypeName'].value_counts()

,count
GovernmentProcedureTypeName,
יבוא מסחרי,1502102
יבוא מסחרי-בלדרים,349765
יבוא מסחרי-אוטונומיה,120477
יבוא אישי,41468
יבוא אישי-בלדרים,36704
"יבוא מסחרי-שח""מ",13147
יבוא מסחרי -מסירה ישירה,4447
יבוא מסחרי-אוטונומיה-בלדרים,3457
"יבוא מסחרי-שח""מ-אוטונומיה",2789


In [326]:
work_df['GovernmentProcedureTypeEnglishName'].value_counts()

,count
GovernmentProcedureTypeEnglishName,
CommercialImport,1502102
CommercialImportCourier,349765
CommercialImportAutonomy,120477
HomeUseImport,41468
HomeUseImportCourier,36704
CommercialImportReleaseFromWarehouse,13147
CommercialImportImmediateRelease,4447
CommercialImportAutonomyCourier,3457
CommercialImportReleaseFromWarehouseAutonomy,2789


שלב של הפיכת נתונים גולמיים לתובנות

In [327]:
df_clean=work_df.copy()

In [328]:
#Check for missing values
df_clean.isna().mean().sort_values(ascending=False)

,0
Origin_Country,0.000218
GoodsDescription,0.000080
Month,0.000000
CustomsItem_8_Digits,0.000000
CustomsHouse,0.000000
Quantity_MeasurementUnitName,0.000000
Quantity,0.000000
NISCurrencyAmount,0.000000
GovernmentProcedureTypeName,0.000000
GovernmentProcedureTypeEnglishName,0.000000


In [329]:
df_clean = df_clean[df_clean["Origin_Country"].notna()].copy()
df_clean = df_clean[df_clean["GoodsDescription"].notna()].copy()

In [330]:
df_clean.isna().mean().sort_values(ascending=False)

,0
Month,0.0
Origin_Country,0.0
CustomsItem_8_Digits,0.0
CustomsHouse,0.0
Quantity,0.0
Quantity_MeasurementUnitName,0.0
NISCurrencyAmount,0.0
GovernmentProcedureTypeName,0.0
GovernmentProcedureTypeEnglishName,0.0
GoodsDescription,0.0


In [331]:
#Checking for negative/zero values
(df_clean["Quantity"] <= 0).sum(), (df_clean["NISCurrencyAmount"] <= 0).sum()

(np.int64(0), np.int64(33))

In [332]:
negatives=df_clean[df_clean["NISCurrencyAmount"] <= 0]
negatives

,Month,Origin_Country,CustomsItem_8_Digits,CustomsHouse,Quantity,Quantity_MeasurementUnitName,NISCurrencyAmount,GovernmentProcedureTypeName,GovernmentProcedureTypeEnglishName,GoodsDescription,FullGoodsDescription
205743,2,IN,49019990,Ben Gurion Airport,1.00,Kilogram,0.0,יבוא מסחרי-אוטונומיה-בלדרים,CommercialImportAutonomyCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."
264762,1,VN,49019990,Ben Gurion Airport,1.00,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."
276968,2,VN,49019990,Ben Gurion Airport,3.00,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."
432592,3,VN,49019990,Ben Gurion Airport,2.00,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."
432769,4,IN,29054900,Ben Gurion Airport,0.10,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,-- Other,"Acyclic alcohols and their halogenated, sulpho..."
445036,4,IN,29349900,Ben Gurion Airport,0.01,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,-- Other,"Nucleic acids and their salts, whether or not ..."
485955,5,SE,71131190,Ben Gurion Airport,0.05,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,"Articles of jewellery and parts thereof, of pr..."
528675,4,IN,29242990,Ben Gurion Airport,0.01,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,Carboxyamide-function compounds; amide-functio...
528986,4,VN,49019990,Ben Gurion Airport,6.00,Kilogram,0.0,יבוא מסחרי-בלדרים,CommercialImportCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."
529218,4,IN,49019990,Ben Gurion Airport,1.00,Kilogram,0.0,יבוא מסחרי-אוטונומיה-בלדרים,CommercialImportAutonomyCourier,---OTHERS,"Printed books, brochures, leaflets and similar..."


In [333]:
df_clean = df_clean[df_clean["NISCurrencyAmount"] > 0].copy()

In [334]:
#Checking a variety of measurement units
df_clean["Quantity_MeasurementUnitName"].value_counts()

,count
Quantity_MeasurementUnitName,
Kilogram,1048177
Each,958046
Pair,25297
square metre,24214
Litre,11649
1000 Litres / Cubic Metre,7140
Tonne,3235
1000 Un.,2222
Carat,1545


In [335]:
df_clean["Origin_Country"].value_counts()

,count
Origin_Country,
CN,369412
US,205678
DE,190865
IT,139050
GB,79733
...,...
VC,2
GL,1
GS,1


In [ ]:
output_path = r"C:\imports_data\imports_2024_clean.csv"
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")

print("הקובץ נשמר בהצלחה ב–", output_path)

In [ ]:
df_clean.sample(20000).to_csv("imports_sample.csv", index=False, encoding="utf-8-sig")

Leading import categories by monetary value throughout 2025

In [ ]:
df_clean.groupby(
    ["GovernmentProcedureTypeEnglishName","GoodsDescription","FullGoodsDescription"]
)["NISCurrencyAmount"].sum().sort_values(ascending=False).head(20)


In [ ]:
#Amount by procedure
ProcedureType = (
    df_clean.groupby("GovernmentProcedureTypeEnglishName")["NISCurrencyAmount"]
          .sum()
          .sort_values(ascending=False)
          .reset_index()
)

ProcedureType['Bilion_NIS']=ProcedureType['NISCurrencyAmount']/1e9

#Division into Top 4 + Others
top5 = ProcedureType.head(4).copy()

others_sum = ProcedureType.iloc[4:]["Bilion_NIS"].sum()

# Add a record to "Other procedures"
others_row = pd.DataFrame({
    "GovernmentProcedureTypeEnglishName": ["Others"],
    "Bilion_NIS": [others_sum]
})

pie_df = pd.concat([top5, others_row], ignore_index=True)

#Pie Graph
fig = px.pie(
    pie_df,
    values="Bilion_NIS",
    names="GovernmentProcedureTypeEnglishName",
    title="Goverment Procedure Incomes",
    hover_data=["Bilion_NIS"]
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.show()


**seasonality and trend analysis on Israel’s 2024 import data.**
The main goal is finding:

* monthly fluctuations

* seasonal patterns

* anomalies

* trend changes

* which countries/products drive seasonality

In [ ]:

#Find the top 20 countries by total money
top_countries = (
    df_clean.groupby("Origin_Country")["NISCurrencyAmount"]
            .sum()
            .sort_values(ascending=False)
            .head(20)
            .index
)

# Filter only to the top countries
df_top = df_clean[df_clean["Origin_Country"].isin(top_countries)].copy()

# Sum by country + month
monthly_country = (
    df_top
    .groupby(["Origin_Country", "Month"], as_index=False)["NISCurrencyAmount"]
    .sum()
)
#Converted to millions for ease of viewing
monthly_country["NIS_Million"] = monthly_country["NISCurrencyAmount"] / 1e6

# Create heatmap
fig = px.density_heatmap(
    monthly_country,
    x="Month",
    y="Origin_Country",
    z="NIS_Million",
    color_continuous_scale="Sunsetdark",
    labels={
        "Month": "Between Month",
        "Origin_Country": "Origin country",
        "NIS_Million": "Import value (Million NIS)"
    },
    title="Monthly Imports by Top 20 Origin Countries (Million NIS)"
)

fig.update_layout(
    xaxis=dict(dtick=1),
    margin=dict(t=50, l=80, r=40, b=40)
)

fig.show()


In [ ]:
monthly_imports = (
    df_clean.groupby("Month")["NISCurrencyAmount"]
            .sum()
            .reset_index()
)

# Convert to Millions
monthly_imports["NIS_Million"] = monthly_imports["NISCurrencyAmount"] / 1e6

fig = px.bar(
    monthly_imports,
    x="Month",
    y="NIS_Million",
    color="NIS_Million",   #color by total value
    color_continuous_scale="Inferno",
    labels={"NIS_Million": "Million NIS", "Month": "Month"},
    title="Monthly Imports to Israel (Million NIS)"
)

fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis_title="Million NIS"
)

fig.show()

Simple forecasting using smoothing methods

In [ ]:
#Prepare the univariate time series
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

monthly_ts = (
    df_clean.groupby("Month")["NISCurrencyAmount"]
    .sum()
    .sort_index()
    .reset_index()
)
monthly_ts["NIS_Million"] = monthly_ts["NISCurrencyAmount"] / 1e6

ts = monthly_ts.set_index("Month")["NIS_Million"]
ts

In [ ]:
#We aggregate total imports by month:
monthly_ts = (
    df_clean.groupby("Month")["NISCurrencyAmount"]
    .sum()
    .sort_index()
    .reset_index()
)

monthly_ts["NIS_Million"] = monthly_ts["NISCurrencyAmount"] / 1e6
monthly_ts

#This identifies monthly dynamics:
monthly_ts["MoM_percent_change"] = (
    monthly_ts["NIS_Million"].pct_change() * 100
)
monthly_ts

In [ ]:
monthly_ts["Rolling_3M"] = monthly_ts["NIS_Million"].rolling(3).mean()
monthly_ts["Rolling_2M"] = monthly_ts["NIS_Million"].rolling(2).mean()

fig = px.line(
    monthly_ts,
    x="Month",
    y=["NIS_Million", "Rolling_3M"],
    markers=True,
    title="Monthly Imports – Trend + 3-Month Moving Average"
)
fig.update_layout(xaxis=dict(dtick=1))
fig.show()


Because the dataset includes only one year of monthly import data, full seasonal decomposition is not possible. Instead, the analysis relies on month-over-month changes, moving averages, and standardized deviations (z-scores) to uncover patterns. These methods reveal meaningful fluctuations within the year, highlight peak import months, and identify potential anomalies, even without multiple years of historical data.

In [ ]:
#Visualization: Month-over-Month Changes
fig = px.bar(
    monthly_ts,
    x="Month",
    y="MoM_percent_change",
    title="Month-over-Month Import Change (%)",
    labels={"MoM_percent_change": "% Change"}
)
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

In [ ]:
monthly_ts["Z_score"] = (
    (monthly_ts["NIS_Million"] - monthly_ts["NIS_Million"].mean())
    / monthly_ts["NIS_Million"].std()
)

fig = px.bar(
    monthly_ts,
    x="Month",
    y="Z_score",
    title="Relative Strength of Monthly Imports (Z-Score)",
    labels={"Z_score": "Z-score"}
)
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

In [ ]:
#Seasonal Analysis by Country
monthly_country = (
    df_clean.groupby(["Month", "Origin_Country"])["NISCurrencyAmount"]
    .sum()
    .reset_index()
)

monthly_country["NIS_Million"] = monthly_country["NISCurrencyAmount"] / 1e6

top_countries = (
    df_clean.groupby("Origin_Country")["NISCurrencyAmount"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

fig = px.line(
    monthly_country[monthly_country["Origin_Country"].isin(top_countries)],
    x="Month",
    y="NIS_Million",
    color="Origin_Country",
    markers=True,
    title="Seasonality – Top 5 Import Countries"
)
fig.update_layout(xaxis=dict(dtick=1))
fig.show()


# Anomaly detection with time-series anomaly method Forest

**Goal:** detect unusual country–month combinations among the top 5 countries, based on import volume.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# --- 1. Find top 5 countries by total annual import ---
top5_countries = (
    monthly_country.groupby("Origin_Country")["NIS_Million"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# --- 2. Filter dataframe to include only those 5 countries ---
df_z = monthly_country[monthly_country["Origin_Country"].isin(top5_countries)].copy()

# --- 3. Compute Z-score per country ---
df_z["Z"] = (
    df_z.groupby("Origin_Country")["NIS_Million"]
    .transform(lambda x: (x - x.mean()) / x.std())
)

# Replace NaN Z-scores (happens if std=0)
df_z["Z"] = df_z["Z"].fillna(0)

# --- 4. Mark anomalies ---
df_z["Anomaly"] = df_z["Z"].apply(lambda z: "Anomaly" if abs(z) > 2.0 else "Normal")

# --- 5. Create positive size variable ---
df_z["Abs_Z"] = df_z["Z"].abs() + 0.1   # prevents zero or negative size

# --- 6. Build the scatter plot ---
fig = px.scatter(
    df_z,
    x="Month",
    y="NIS_Million",
    color="Origin_Country",
    symbol="Anomaly",
    size="Abs_Z",
    size_max=30,
    symbol_map={"Normal":"circle", "Anomaly":"diamond"},
    title="Z-Score Based Anomaly Detection – Top 5 Countries",
    hover_data=["Origin_Country", "NIS_Million", "Z"],
)

# --- 7. Emphasize anomalies ---
fig.update_traces(
    selector=dict(marker_symbol="diamond"),
    marker=dict(
        line=dict(width=3, color="black"),
        color="red",
        opacity=0.95
    )
)

# --- 8. Make normal points more subtle ---
fig.update_traces(
    selector=dict(marker_symbol="circle"),
    marker=dict(
        opacity=0.6
    )
)

fig.update_layout(
    xaxis=dict(dtick=1),
    legend_title_text="Country & Anomaly Status"
)

fig.show()


The anomaly analysis, based on Z-score standardization within each of the top five origin countries, highlights several months with unusually high or low import levels relative to the typical annual pattern. Since Z-score compares each month to the country’s own distribution, it effectively normalizes differences in scale.

The most prominent anomalies appear in China and Germany, where imports in month 12 are significantly above trend, while Turkey shows sharp downward deviations in months 7–8. These anomalies likely correspond to one-off bulk shipments, seasonal procurement cycles, or supply disruptions. Overlaying rolling-trend lines confirms that these points deviate meaningfully from normal monthly variation, indicating genuine irregularities rather than noise.

# Approach: Country-level anomaly detection

We will detect anomalies by analyzing each country’s overall yearly profile, instead of individual monthly points.

Each country is represented by a feature vector capturing its characteristics:

* Features used per country

* Total annual import value

* Average monthly import

* Maximum monthly import

* Volatility (standard deviation)

* Coefficient of variation (volatility relative to average)

* Growth patterns (simple month 12 vs month 1 change)

These features are fed into an anomaly detector such as:

* Z-score

* Isolation Forest

* LOF (Local Outlier Factor)

* DBSCAN (cluster anomaly)


In [ ]:
#Construct country-level feature table

# Monthly totals by country
country_month = (
    df_clean
    .groupby(["Origin_Country", "Month"])["NISCurrencyAmount"]
    .sum()
    .reset_index()
)

# Annual totals
total_by_country = (
    country_month
    .groupby("Origin_Country")["NISCurrencyAmount"]
    .sum()
    .rename("TotalAnnual")
)

# Average monthly
avg_monthly = (
    country_month
    .groupby("Origin_Country")["NISCurrencyAmount"]
    .mean()
    .rename("AvgMonthly")
)

# Max monthly
max_monthly = (
    country_month
    .groupby("Origin_Country")["NISCurrencyAmount"]
    .max()
    .rename("MaxMonthly")
)

# Volatility (std)
std_monthly = (
    country_month
    .groupby("Origin_Country")["NISCurrencyAmount"]
    .std()
    .rename("Volatility")
)

# Coefficient of variation (std/mean)
cv = (std_monthly / avg_monthly).rename("CV")

# Growth from month 1 → month 12
c_growth = country_month.pivot(index="Origin_Country", columns="Month", values="NISCurrencyAmount")
c_growth["Growth12"] = (c_growth[12] - c_growth[1]) / c_growth[1]

# Combine feature set
country_features = pd.concat(
    [total_by_country, avg_monthly, max_monthly, std_monthly, cv, c_growth["Growth12"]],
    axis=1
).fillna(0)

country_features.head()


In [ ]:
#Normalize and apply anomaly detection (Z-score)
# Standardize features for fairness
from sklearn.preprocessing import StandardScaler

scaled = StandardScaler().fit_transform(country_features)
country_scaled = pd.DataFrame(scaled, index=country_features.index, columns=country_features.columns)

# Compute anomaly score = distance from mean
country_scaled["AnomalyScore"] = np.sqrt((country_scaled**2).sum(axis=1))

# Top anomalous countries (highest distances)
anomalous_countries = country_scaled.sort_values("AnomalyScore", ascending=False).head(10)

anomalous_countries


In [ ]:
fig = px.scatter(
    country_scaled.reset_index(),
    x="TotalAnnual",
    y="Volatility",
    size="AnomalyScore", # Changed from 'MaxMonthly' to 'AnomalyScore'
    color="AnomalyScore",
    hover_name="Origin_Country",
    title="Country-Level Anomaly Map (Value vs Volatility vs Outlier Score)",
    labels={"TotalAnnual": "Total Annual Imports", "Volatility": "Volatility (standard deviation)"},
    color_continuous_scale="Sunsetdark"
)

fig.show()

Centralization and Dependency - How dependent is Israel on countries/products?

In [ ]:
total_imports = df_clean["NISCurrencyAmount"].sum()

country_share = (
    df_clean.groupby("Origin_Country")["NISCurrencyAmount"]
    .sum()
    .reset_index()
)

country_share["SharePercent"] = 100 * country_share["NISCurrencyAmount"] / total_imports
country_share.sort_values("SharePercent", ascending=False).head(10)


In [ ]:
origin_country_imports = (
    df_clean.groupby("Origin_Country")["NISCurrencyAmount"]
    .sum()
    .sort_values(ascending=False) # Removed the 'NISCurrencyAmount' argument
    .head(20)
    .reset_index() # Added .reset_index() to convert Series to DataFrame
)

top10 = origin_country_imports.head(10).copy() # Added .copy() to avoid SettingWithCopyWarning


top10["NIS_Billion"] = top10["NISCurrencyAmount"] / 1e9 # Changed to use top10 directly

fig = px.bar(
    top10,
    x="Origin_Country",
    y="NIS_Billion",
    color="NIS_Billion",
    color_continuous_scale="Inferno",
    labels={"NIS_Billion": "Billion NIS", "Origin_Country": "Origin Country"},
    title="TOP Origin country Imports to Israel in 2024(Million NIS)"
)

fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis_title="Billion NIS"
)

fig.show()

In [ ]:
# Sum by country
ProcedureType = (
    df_clean.groupby("Origin_Country")["NISCurrencyAmount"]
          .sum()
          .sort_values(ascending=False)
          .reset_index()
)

ProcedureType['Bilion_NIS']=ProcedureType['NISCurrencyAmount']/1e9


top5 = ProcedureType.head(9).copy()

others_sum = ProcedureType.iloc[9:]["Bilion_NIS"].sum()

others_row = pd.DataFrame({
    "Origin_Country": ["Others"],
    "Bilion_NIS": [others_sum]
})

pie_df = pd.concat([top5, others_row], ignore_index=True)

fig = px.pie(
    pie_df,
    values="Bilion_NIS",
    names="Origin_Country",
    title="NIS by origin country",
    hover_data=["Bilion_NIS"]
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.show()

Product concentration: check whether there are products that only come from certain countries.

In [ ]:
prod_country = (
    df_clean
    .groupby(["GoodsDescription","FullGoodsDescription", "Origin_Country"])["NISCurrencyAmount"]
    .sum()
    .reset_index()
)

country_count = (
    prod_country.groupby("FullGoodsDescription")["Origin_Country"]
    .nunique()
    .reset_index(name="country_count")
)

eligible_products = (
    country_count[country_count["country_count"] >= 5]["FullGoodsDescription"]
)
prod_filtered = prod_country[
    prod_country["FullGoodsDescription"].isin(eligible_products)
]

def top3_share(group):
    total = group["NISCurrencyAmount"].sum()
    top3_sum = group.sort_values("NISCurrencyAmount", ascending=False).head(3)["NISCurrencyAmount"].sum()
    return (top3_sum / total) * 100

prod_concentration = (
    prod_filtered
    .groupby("FullGoodsDescription")
    .apply(top3_share)
    .reset_index(name="Top3SharePercent")
    .sort_values("Top3SharePercent", ascending=False)
)
prod_concentration.head(10)

In [ ]:
top_concentrated = prod_concentration.head(10)
top_concentrated


In [ ]:
product_totals = (
    df_clean.groupby("FullGoodsDescription")["NISCurrencyAmount"]
    .sum()
    .reset_index(name="TotalImportNIS")
)

top_concentrated = top_concentrated.merge(
    product_totals, on="FullGoodsDescription", how="left"
)

top_concentrated.head()

In [ ]:
product = top_concentrated.iloc[0]["FullGoodsDescription"]
product


In [ ]:
df_clean[df_clean["FullGoodsDescription"] == product] \
    .groupby("Origin_Country")["NISCurrencyAmount"] \
    .sum() \
    .sort_values(ascending=False) \
    .reset_index()

In [ ]:
def inspect_product(full_desc):
    print("\n==============================")
    print("Product:", full_desc)
    print("==============================\n")

    # סכום כולל
    total = df_clean[df_clean["FullGoodsDescription"] == full_desc]["NISCurrencyAmount"].sum()
    print(f"Total import value: {total:,.0f} NIS\n")

    # מדינות
    country_breakdown = (
        df_clean[df_clean["FullGoodsDescription"] == full_desc]
        .groupby("Origin_Country")["NISCurrencyAmount"]
        .sum()
        .sort_values(ascending=False)
        .reset_index()
    )

    display(country_breakdown)

    # אחוזים
    country_breakdown["Percent"] = 100 * country_breakdown["NISCurrencyAmount"] / total
    print("\nCountry share (%):\n")
    display(country_breakdown)


In [ ]:
inspect_product(top_concentrated.iloc[0]["FullGoodsDescription"])
inspect_product(top_concentrated.iloc[1]["FullGoodsDescription"])
inspect_product(top_concentrated.iloc[2]["FullGoodsDescription"])


In [ ]:
for desc in top_concentrated["FullGoodsDescription"].head(10):
    inspect_product(desc)

In [ ]:
# Merge concentration with total import values
product_concentration = prod_concentration.merge(
    product_totals,
    on="FullGoodsDescription",
    how="left"
)

# Convert to millions for readability
product_concentration["TotalImportMillion"] = product_concentration["TotalImportNIS"] / 1e6

product_concentration_top = (
    product_concentration
    .sort_values("TotalImportNIS", ascending=False)
    .head(50)     # adjust as needed
)

fig = px.treemap(
    product_concentration_top,    # or product_concentration
    path=[px.Constant("Products"), "FullGoodsDescription"],
    values="TotalImportMillion",
    color="Top3SharePercent",
    color_continuous_scale="Temps",
    hover_data={
        "Top3SharePercent": ':.2f',
        "TotalImportMillion": ':.2f'
    }
)

fig.update_layout(
    title="Product Concentration: Import Value vs Supply Concentration",
    margin=dict(t=50, l=25, r=25, b=25),
    coloraxis_colorbar=dict(
        title="Top 3 Share (%)"
    )
)

fig.show()

In [ ]:
# Total import by product–country
prod_country = (
    df_clean
    .groupby(["FullGoodsDescription", "Origin_Country"])["NISCurrencyAmount"]
    .sum()
    .reset_index()
)

# Total import per product
product_totals = (
    prod_country
    .groupby("FullGoodsDescription")["NISCurrencyAmount"]
    .sum()
    .reset_index(name="ProductTotalNIS")
)

# Keep only the 100 most valuable products
top100_products = (
    product_totals
    .sort_values("ProductTotalNIS", ascending=False)
    .head(100)
)

top100_list = top100_products["FullGoodsDescription"].tolist()
prod_country_top100 = prod_country[
    prod_country["FullGoodsDescription"].isin(top100_list)
].copy()

# Sort so that top 3 per product are first
prod_country_sorted = prod_country_top100.sort_values(
    ["FullGoodsDescription", "NISCurrencyAmount"],
    ascending=[True, False]
)

# Top 3 countries per product (within the top 100 products)
top3 = (
    prod_country_sorted
    .groupby("FullGoodsDescription")
    .head(3)
    .copy()
)
# Merge total per product (already computed in product_totals)
top3 = top3.merge(
    product_totals,
    on="FullGoodsDescription",
    how="left"
)

# Share of each country out of the whole product
top3["ShareOfProduct"] = 100 * top3["NISCurrencyAmount"] / top3["ProductTotalNIS"]

# Share within the top-3 block (optional, sums to 100% per product)
top3_totals = (
    top3.groupby("FullGoodsDescription")["NISCurrencyAmount"]
    .sum()
    .reset_index(name="Top3TotalNIS")
)

top3 = top3.merge(top3_totals, on="FullGoodsDescription", how="left")
top3["ShareWithinTop3"] = 100 * top3["NISCurrencyAmount"] / top3["Top3TotalNIS"]
import plotly.express as px

fig = px.treemap(
    top3,
    path=[px.Constant("Top 100 Products"), "FullGoodsDescription", "Origin_Country"],
    values="NISCurrencyAmount",
    color="ShareOfProduct",
    color_continuous_scale="Purpor",
    custom_data=["ShareOfProduct", "ShareWithinTop3", "ProductTotalNIS"]
)

fig.update_traces(
    texttemplate="<b>%{label}</b><br>%{customdata[0]:.1f}% of product",
    textfont=dict(size=11)
)

fig.update_layout(
    title="Top 100 Imported Products – Top 3 Supplying Countries and Concentration",
    margin=dict(t=50, l=25, r=25, b=25),
    coloraxis_colorbar=dict(
        title="Share of product<br>(Top-3 country %)"
    )
)

fig.show()


The analysis reveals that Israel’s top imported products are structurally concentrated, with the majority of categories relying heavily on one or two countries. For many high-value goods, the top supplier accounts for 80–95% of total import value, indicating substantial dependency and potential supply-chain vulnerability. Only a small portion of products show balanced multi-country sourcing. These findings highlight the importance of monitoring supplier risk, identifying alternative production markets, and prioritizing diversification for critical categories.

**Combined product value + concentration reveals the highest-risk items**

The treemap makes it easy to identify “high value + high concentration” cases —
a critical combination for risk assessment.

Products in the upper-left (large tiles, dark colors) represent:

*  high import value (major economic impact)

*  high concentration (major risk exposure)

These are ideal candidates for:

*  supplier diversification

*  strategic inventory

*  alternative sourcing policies

*  long-term procurement planning

In [ ]:
# Merge concentration with total import values
risk_table = prod_concentration.merge(
    product_totals,
    on="FullGoodsDescription",
    how="left"
)

# Convert to millions to make the table readable
risk_table["TotalImportMillion"] = risk_table["ProductTotalNIS"] / 1e6

# Compute risk score = high value × high dependency
risk_table["RiskScore"] = (
    risk_table["TotalImportMillion"] * (risk_table["Top3SharePercent"] / 100)
)

# Sort descending: highest risk first
highest_risk = (
    risk_table
    .sort_values("RiskScore", ascending=False)
    .head(20)      # top 20 highest-risk products
)

highest_risk = highest_risk[
    ["FullGoodsDescription",
     "TotalImportMillion",
     "Top3SharePercent",
     "RiskScore"]
]

highest_risk.reset_index(drop=True, inplace=True)
highest_risk

The ranking shows that several high-value import categories are also among the most supply-concentrated. For these products, over 80–95% of the import value originates from just one or two countries. As a result, the economic exposure to supply disruptions is significant. Products at the top of the list represent the highest risk due to the combination of large import volumes and near-monopolistic supply structures. These items should be prioritized for diversification efforts or strategic stock planning.

Crude oil import data analysis

In [ ]:
#Filter crude oil records
oil_df = df_clean[df_clean["FullGoodsDescription"]
            .str.contains("Petroleum oils", case=False, na=False)
           ].copy()

oil_df.head()
oil_df.shape


In [ ]:
oil_df.head()

In [ ]:
#Countries from which crude oil comes to Israel in monetary units
oil_by_country = (
    oil_df.groupby("Origin_Country")["NISCurrencyAmount"]
    .sum()
    .sort_values(ascending=False)
    .head(15)
)


In [ ]:
import plotly.express as px


oil_by_country = (
    oil_df.groupby("Origin_Country")["NISCurrencyAmount"]
          .sum()
          .sort_values(ascending=False)
          .reset_index()
)

top5 = oil_by_country.head(4).copy()

others_sum = oil_by_country.iloc[4:]["NISCurrencyAmount"].sum()

others_row = pd.DataFrame({
    "Origin_Country": ["Other Countries"],
    "NISCurrencyAmount": [others_sum]
})

pie_df = pd.concat([top5, others_row], ignore_index=True)

fig = px.pie(
    pie_df,
    values="NISCurrencyAmount",
    names="Origin_Country",
    title="Top 5 Crude Oil Exporters to Israel (NIS)",
    hover_data=["NISCurrencyAmount"]
)

fig.update_traces(
    textposition='outside',
    textinfo='percent+label'
)

fig.show()


In order to calculate a price per unit, I will make sure that the units of measure

In [ ]:
oil_df['Quantity_MeasurementUnitName'].value_counts()

In [ ]:
# We will only fix the records in concrete
mask_ton = oil_df["Quantity_MeasurementUnitName"] == "Tonne"

oil_df.loc[mask_ton, "Quantity"] = oil_df.loc[mask_ton, "Quantity"] * 1000
oil_df.loc[mask_ton, "Quantity_MeasurementUnitName"] = "Kilogram"
oil_df['Quantity_MeasurementUnitName'].value_counts()

In [ ]:
# To convert to tons, we will use an average density of 0.85 tons/cubic meter (common in crude oil) and then multiply by 1000 to convert to kilograms.
mask_l_c = oil_df["Quantity_MeasurementUnitName"] == "1000 Litres / Cubic Metre"

oil_df.loc[mask_l_c, "Quantity"] = oil_df.loc[mask_l_c, "Quantity"] * 1000 * 0.85
oil_df.loc[mask_l_c, "Quantity_MeasurementUnitName"] = "Kilogram"
oil_df['Quantity_MeasurementUnitName'].value_counts()

In [ ]:
oil_df.info()

In [ ]:
#Oil import trend by month (quantity and money)
oil_month_qty = (
    oil_df.groupby("Month")["Quantity"]
    .sum()
)

oil_month_value = (
    oil_df.groupby("Month")["NISCurrencyAmount"]
    .sum()
)

print(oil_month_qty.head())
print(oil_month_value.head())

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a figure with two graphs (2 rows, 1 column)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=("Monthly Crude Oil Quantity", "Monthly Crude Oil Value (NIS)")
)

# Bar graph – quantity
fig.add_trace(
    go.Bar(
        x=oil_month_qty.index, # Access the index which contains the month numbers
        y=oil_month_qty, # Use the Series directly
        name="Quantity (kg)"
    ),
    row=1, col=1
)

# Bar graph – money
fig.add_trace(
    go.Bar(
        x=oil_month_value.index, # Access the index which contains the month numbers
        y=oil_month_value, # Use the Series directly
        name="Value (NIS)"
    ),
    row=2, col=1
)

fig.update_layout(
    height=700,
    title_text="Monthly Crude Oil Imports – Quantity and Value",
    showlegend=False
)

# # Shared X-axis – months 1–12
fig.update_xaxes(title_text="Month", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="Quantity (kg)", row=1, col=1)
fig.update_yaxes(title_text="NIS", row=2, col=1)

fig.show()

We will check whether there is noise in the price feature.

In [ ]:
oil_df["price_per_kg"] = oil_df["NISCurrencyAmount"] / oil_df["Quantity"]
oil_df['price_per_kg']

In [ ]:
def detect_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower) | (df[col] > upper)]
    num_outliers = outliers.shape[0]
    total = df.shape[0]
    percent = (num_outliers / total) * 100

    print(f"IQR Outlier detection for {col}")
    print(f"Lower bound: {lower:.2f}")
    print(f"Upper bound: {upper:.2f}")
    print(f"Number of outliers: {num_outliers}")
    print(f"Percent of outliers: {percent:.2f}%")

    return outliers

outliers_ppm = detect_outliers_iqr(oil_df, "price_per_kg")


In [ ]:
Lower_bound=-182.36
Upper_bound=341.65

outliers = oil_df.loc[
    (oil_df['price_per_kg'] < Lower_bound) |
    (oil_df['price_per_kg'] > Upper_bound ), ["price_per_kg","Quantity","NISCurrencyAmount","GoodsDescription","FullGoodsDescription","Origin_Country"]]
outliers

You can see that the price is very high when it comes to small quantities.

In [ ]:
oil_std=oil_df.copy()
Q1 = oil_std["price_per_kg"].quantile(0.25)
Q3 = oil_std["price_per_kg"].quantile(0.75)
IQR = Q3 - Q1

lower = max(0, Q1 - 1.5 * IQR)
upper = Q3 + 1.5 * IQR

oil_clean = oil_std[(oil_std["price_per_kg"] >= lower) & (oil_std["price_per_kg"] <= upper)].copy()


Checking the price difference per kg

In [ ]:
oil_clean["price_per_kg"].describe()

In [ ]:
#Sum by country
oil_country_stats = (
    oil_clean.groupby("Origin_Country")
    .agg(
        total_qty_kg=("Quantity", "sum"),
        avg_price_per_kg=("price_per_kg", "mean"),
        total_value_nis=("NISCurrencyAmount", "sum")
    )
    .reset_index()
)

In [ ]:
# Select the 15 countries with the highest quantity
top_n = 20
top_countries = (
    oil_country_stats.sort_values("total_qty_kg", ascending=False)
    .head(top_n)
)

fig = px.scatter(
    top_countries,
    x="total_qty_kg",
    y="avg_price_per_kg",
    color="Origin_Country",
    size="total_value_nis",
    hover_name="Origin_Country",
    hover_data={
        "total_qty_kg": ":,.0f",
        "avg_price_per_kg": ":.4f",
        "total_value_nis": ":,.0f"
    },
    title=f"Crude Oil: Quantity vs Avg Price per Kg (Top {top_n} Countries)",
    labels={
        "total_qty_kg": "Total Quantity Imported (kg)",
        "avg_price_per_kg": "Average Price per Kg (NIS)"
    }
)

fig.update_traces(marker=dict(opacity=0.8, line=dict(width=1, color="black")))
fig.update_layout(height=600)
fig.show()
